In [1]:
import numpy as np
import scipy.optimize as opt
from scipy.stats import poisson

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Baseline model (single crime fair allocation)

In [15]:
class Baseline:
  def __init__(self, C: np.ndarray, V: int, G:int):
    '''
    input:
    C = distribution of crime. 
        C is 2darray where the row represents C_i.
        Each index of C_i represents the number of crime
        and the value represents frequency.
    V = the total number of units of resources.
    G = the number of regions.

    output:
    W_opt = optimal allocation of V
    X_max = the maximized objective value
    '''
    self.C = C
    self.V = V
    self.G = G
    self.W_opt = np.zeros((self.G))
    self.X_max = 0
    self.f = np.ones((self.G, self.V+1))

  def disc(self, v_i: int, c_i: int):
    '''
    Calculate discovery function.
    Input: 
    v_i = the number of patrol allocated to region i (G_i).
    c_i = the number of crime in region i.
    
    Output:
    number of discovered crime.
    '''
    return min(v_i, c_i)
  
  def calc_f(self, i: int, v_i: int):
    '''
    Calculate f_i(v_i).
    Input:
    v_i = the number of patrol allocated to region i (G_i).
    C_i = discribution of crime in region i.

    Output:
    f_i(v_i) = probability of discovering crime for allocating v_i patrols in region i.
    '''
    self.f[i, v_i] = 0
    lamb = self.C[i]
    for c_i in range(v_i):
      if c_i:
        prob = poisson.pmf(k=c_i, mu=lamb)
        self.f[i, v_i] += (self.disc(v_i, c_i)/c_i)*prob
    return

  def get_j_star(self, v:np.ndarray, ub:np.ndarray):
    '''
    Get j_star.
    Input:
    v = the number of patrol for all region.
    ub = upper bounds for all region.
    '''
    max_val, j_star = -10e9, 0
    for j in range(self.G):
      if v[j] < ub[j]:
        lamb = self.C[j]
        diff_tau = (1-poisson.cdf(v[j], mu=lamb)) - (1-poisson.cdf(v[j]-1, mu=lamb))
        if max_val <= diff_tau:
          max_val = diff_tau
          j_star = j
    return j_star
  
  def found_bound(self, region, alpha, f_i):
    lb = self.V
    ub = 0
    for v in range(self.V):
      if self.f[region, v] == 1:
        self.calc_f(region, v)
      if self.f[region, v] > f_i - alpha and v < lb:
        lb = v
      elif self.f[region, v] >= f_i -alpha and self.f[region, v] <= f_i and v > ub:
        ub = v
    return lb, ub


  def algorithm_1(self, alpha: float):
    '''
    Run algorithm1 that computes an optimal fair allocation
    in the precision model
    Input:
    alpha = fairness threshold [0,1]
    V = the number of patrol for all region.
    '''

    for i in range(self.G):
      v = np.zeros(self.G, dtype=int)
      ub = np.zeros(self.G)
      lb = np.zeros(self.G)
      for v_i in range(self.V+1):
        print('regin: %d, v: %d'%(i, v_i))
        v[i] = v_i
        # compute f_i(v_i).
        if self.f[i, v_i] == 1:
          self.calc_f(i, v_i) 
        ub[i] = v[i] # upper bound on allocation to group i
        lb[i] = v[i] # lower bound on allocation to group i
        for j in range(self.G):
          if i == j:
            continue
          # update lower and upper bounds on allocation to group j
          lb[j], ub[j] = self.found_bound(j, alpha, self.f[i, v_i])
          v[j] = lb[j]
        
        # check whether v exceed #. available patrollers
        if sum(v) > self.V:
          print(sum(v))
          break
        #allocating the remaining patrollers
        V_r = int(self.V - sum(v))
        for t in range(1, V_r+1):
          j_star = self.get_j_star(v, ub)
          v[j_star] += 1
        # calculating the obj value
        curr_obj = 0
        for g in range(self.G):
          for n_vi in range(v[g]):
            curr_obj += 1-poisson.cdf(n_vi, mu=self.C[g])
        #X = sum([np.sum((1-poisson.cdf(c, mu=self.C[i]))) for c in range(v[i]) for i in range(self.G)])
        #print(v)
        #print(X)
        print('current allocation: ', v)
        print('current obj value: ', curr_obj)
        if curr_obj > self.X_max:
          print('Update maximum of X...')
          self.X_max = curr_obj
          for g in range(self.G):
            self.W_opt[g] = v[g]
          print(self.W_opt)
        print(self.W_opt)
    return self.W_opt

## Rawdata

In [16]:
alpha = 0.1 # fairness threshold
# C for BATTERY
C = [3.7402269861286253, 5.075662042875158, 6.796973518284994, 7.498108448928121, 
    5.572509457755359, 8.282471626733923, 7.366960907944515, 6.674653215636822, 
    5.215636822194199, 6.181588902900378, 8.184110970996217, 4.411097099621689, 
    2.5422446406052965, 5.750315258511979, 3.3619167717528375, 2.6557377049180326, 
    3.399747793190416, 3.3354350567465323, 2.0756620428751575, 0, 3.308953341740227, 
    3.490542244640605, 5.851197982345523, 0.005044136191677175]
V = 100 # number of patrol resources
G = 24 # 24 districts
# initiate baseline model
baseline = Baseline(C, V, G)
# run greedy algorithm
baseline.algorithm_1(alpha)
print("Fairness threshold:", alpha)
print("Optimal Allocation :", baseline.W_opt)
print("Maximum value of the objective:", baseline.X_max)
print("prob of arrest:", baseline.f)

regin: 0, v: 0
current allocation:  [ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 78  1  1  1  1]
current obj value:  79.47991035772354
Update maximum of X...
[ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. 78.  1.  1.  1.  1.]
[ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. 78.  1.  1.  1.  1.]
regin: 0, v: 1
current allocation:  [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 77  1  1  1  1]
current obj value:  78.50365906961483
[ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. 78.  1.  1.  1.  1.]
regin: 0, v: 2
current allocation:  [ 2  2  3  4  3  5  4  3  2  3  4  2  1  3  1  1  1  1  1  0  1  1  3 49]
current obj value:  50.5679785032909
[ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. 78.  1.  1.  1.  1.]
regin: 0, v: 3
286
regin: 1, v: 0
current allocation:  [ 1  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 78  1  1  1  1]
current o

In [ ]:
# evaluation


### 주석 달린 버전(이해용)

In [ ]:
class Baseline:
  def __init__(self, alpha: float, C: ?, T:, V: int, G:int):
    '''
    alpha = fairness threshold, [0,1]
    C = candidate distribution
    T = cumulative candidate distribution
    V = number of units of resources
    G = number of groups
    '''
    self.alpha = alpha
    self.C = C
    self.T = T
    self.V = V
    self.G = G
    self.W_opt = np.zeros([])
    self.X_max = 0

  def disc(self, v_i, c_i):
    '''
    Calculate discover function.
    Input: 
    v_i = number of patrol allocated to region i (G_i).
    c_i = number of crime in region i.
    
    Output:
    number of discovered crime.
    '''
    return min(v_i, c_i)
  
  def calc_f(self, v_i: int, C_i: ?):
    '''
    Calculate f_i(v_i).
    Input:
    v_i = number of patrol allocated to region i (G_i).
    C_i = discribution of crime in region i.

    Output:
    f_i(v_i) = probability of discovering crime for allocating v_i patrols in region i.
    '''
    f_all = np.array([self.disc(v_i, c_i)/c_i for c_i in C_i])
    return np.mean(f_all)
  
  def get_j_star(self, T:ndarray, v:ndarray, ub:ndarray):
    '''
    Get j_star.
    Input:
    T = cumulative values of all possible allocation for all region (Pr_{c_i ~ C_i}[c_i >= c]).
    v = number of patrol for all region.
    ub = upper bounds for all region.
    '''
    max_val, j_star = 0, 0
    for j in range(self.G):
      if v[j] < ub[j]:
        max_val = max(T[j][v[j]+1]-T[j][v[j]], max_val)
        j_star = j
    return j_star
  
  def run_greedy(self):
    '''
    Run the greedy algorith for fairness allocation.
    Output:
    w_opt = optimal patrol allocation for every region.
    '''
    # guess for group with the highest probability of discovery
    for i in range(self.G): ## G는 뭐냐... 숫자 맞음? 맞는듯. 지역 i를 고려한다.
      v = np.zeros(len(self.G)) # v를 reset한다.. 매 지역마다 리셋..? 맞음.
      ub = np.zeros(len(self.G)) # 각 지역의 patrol 수의 upper bound
      lb = np.zeros(len(self.G)) # 각 지역의 patrol 수의 lower bound
      for v_i in range(V+1): # v_i i번째 지역에 patrol 인원을 그냥 되는데로 다 넣어본다.
        v[i] = v_i
        # compute f_i(v_i). 리스트로 만들어야하나?
        f_i = self.calc_f(v_i, C_i) # f_i 그러니까 지역 i에 v_i명의 patrol을 배치했을 때 그 지역에서 범죄를 발견할 확률을 구한다.
        ub[i] = v_i # upper bound on allocation to group i
        lb[i] = v_i # lower bound on allocation to group i
        for j in self.G and j != i: # 그럼 이제 지역 i가 아닌 다른 모든 지역을 돌아보자.
          # update lower and upper bounds on allocation to group j
          ub[j] = f_i # 또는 f_i - self.alpha/2
          lb[j] = f_i - self.alpha # 또는 f_i - self.alpha/2
          v[j] = lb[j]
        print("Allocated result:", v)
        if sum(v) > V:
          continue
        V_r = V - sum(v_i) # 남는 인력 계산
        for t in range(1, V_r+1): # 남는 인력에 대해 하나씩 분배
          j_star = get_j_star(T, v, ub) # 어느 지역에 할당해줄지 정함.
          v_j_star += 1 # 할당
        X = sum([np.sum(T[:v_i]) for v_i in v]) # 할당이 모두 끝난 뒤 utility를 계산.
        if X > X_max: # utility가 원래 max 값보다 크면,
          print('Update maximum of X...')
          X_max = X # utility의 max 업데이트
          w_opt = ㅍ # allocation 을 업데이트
    return w_opt

  def poisson_logL(self):
    '''
    Calculate loglikelihood of poisson distribution
    Input:

    Output:
    '''
    return

  def solve_mle(self, h: tuple, lamb_min, lamb_max):
    '''
    Find optimal parameter lambda by solving MLE.
    Input:
    h = h_i(t+1). A tuple consists of (v_i(t), o_i(t)).
    Output:
    lamb = An optimal parameter between lamb_min and lamb_max.
    '''
    o, v = h[0], h[1]
    if o < v:
      res = opt.minimize(
      fun=lambda o, lamb: -np.log(poisson.pmf(k=o, mu=lamb)),
      x0=np.array([0.5]), args=(o,), method='BFGS')
      lamb = np.exp(res.x)
    else:
      res = opt.minimize(
      fun=lambda o, lamb: -np.log(poisson.cdf(k=o, mu=lamb)),
      x0=np.array([0.5]), args=(o,), method='BFGS')
      lamb = np.exp(res.x)
    return lamb

  def fair_allocation(self, alpha: float, T: int):
    '''
    Learning an optimal fair allocation.
    Input:
    alpha = fairness threshold. [0,1]
    T = total number of rounds
    Output:
    w_opt = optimal patrol allocation for every region.
    '''
    v = np.zeros([T+1, self.G])
    o = np.zeros([T+1, self.G])
    h = np.zeros([T+1, self.G])
    lamb = np.zeros([T+1, self.G])
    # allocate uniformly
    v[0] = np.array([self.V/self.G]*self.G)
    for t in range(1, T+1):
      # check whether every group is allocated a resource
      if np.any(v[t]):
        # assign the allocation from the previous round
        v[t] = v[t-1]
      # observe o_i(t) for each group.
      o[t] = np.minimum(v[t], c[t])
      for i in range(self.G): # 각 지역마다
        # update history h_i(t+1) with o_i(t) and v_i(t).
        h[t+1,i] = (v[t,i], o[t,i])
        # solve the maximum likelihood estimation problem
        lamb[t, i] = self.solve_mle(h[t+1, i])
      # compute an allocation to be deployed in the next round
      self.C = lamb[t]
      v[t+1] = self.algorithm_1(alpha, self.C, self.V)
    return v[T+1], lamb[T]


# Our model (multi-crime fair allocation)